# Reconocimiento de dígitos con KNN

Veamos como interactuar con las clases implementadas en el módulo de C++ haciendo una experimentación muy simplificada, y a modo de ejemplo, del método KNN para reconocer digitos

## Compilando el módulo desde C++

Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [ ]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

In [ ]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import metnum

## Explorando los datos

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_csv("../data/train.csv")

Es un csv, con columnas por cada pixel

In [ ]:
df_train[:10]

### Reducir el dataset

Ya que el dataset es grande, me quedo con una porción de éste

**<font color=red>ESTO ES SOLO PARA HACER PRUEBAS. CUANDO REPORTEN LOS RESULTADOS, USAR DATASET COMPLETO</font>**

In [ ]:
df_train = df_train[:5000]

En el primer caracter está el dígito a reconocer. Llamamos a esto `y_train`

In [ ]:
# Uso values para mandar todo a arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

X.shape, y.shape

Convierto el resto a matrices de 28 x 28

A ver cómo se ven...

In [ ]:
import matplotlib.pyplot as plt

num = 1917

print(f"Supuestamente es un {int(y[num])}")

img = X[num].reshape(28, 28)
plt.imshow(img, cmap="Greys")


### Balance de clases

¿Tenemos la misma cantidad de cada número?

In [ ]:
df_train["label"].value_counts(normalize=True)

No: tenemos más 1 que 5, pero tampoco está tan desbalanceado

### Conjunto de development

In [ ]:
limit = int(0.8 * X.shape[0]) 

X_train, y_train = X[:limit], y[:limit]
X_val, y_val = X[limit:], y[limit:]

assert len(X_train) == len(y_train)
assert len(X_val) == len(y_val)

print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_val)} de validación")


In [ ]:
pca = metnum.PCA(13)
pca.fit(X_train)
X_train = pca.transform(X_train)

pca.fit(X_val)
X_val = pca.transform(X_val)

clf_metnum = metnum.KNNClassifier(18)
clf_metnum.fit(X_train, y_train)


In [ ]:
%%time
from sklearn.metrics import accuracy_score
acc = accuracy_score(clf_metnum.predict(X_val), y_val)
print("Accuracy: {}".format(acc))


0.914 es el resultado solamente con knn